In [1]:
import numpy as np
import pylab as plt
from treegp import AnisotropicRBF
import treecorr
import ipywidgets as widgets
from ipywidgets import interact
import itertools
import pickle
%matplotlib inline

Here I am generating a 2D gaussian random field (I am using the `make_2d_grf` to reproduce the following equation): 
    
$y \sim {\cal{N}}(0, K)$

where $K$ is the kernel. For this specific example I will used the classical RBF kernel (aka squarred exponential | gaussian kernel):

$K(\vec{X}_i, \vec{X}_j) = \sigma^2 \exp\left( - \frac{1}{2} (\vec{X}_i - \vec{X}_j)^T \textbf{L}^{-1} (\vec{X}_i - \vec{X}_j)\right)$

Where $\textbf{L}$ is the correlation length matrix, and could be represented in general as a 2D symetric positiv definit matrix: 

$\textbf{L} = \begin{pmatrix}
{l_x}^2 & l_{xy} \\
l_{xy} & {l_y}^2
\end{pmatrix}$

If the Gaussian random field is isotropic, then: 

$l_x = l_y,$ and $l_{xy}=0$

I am using `get_correlation_length_matrix`, to generate $\textbf{L}$ and I am using an other parametrization than $l_x, l_y,$ and $l_{xy}$. Instead I am using $l$, $e_1$, and $e_2$ which are mathematicaly equivalent to galaxy shape parameters used in cosmic shear. For isotropic case $l>0$ and $e_1=e_2=0$


In [2]:
def make_2d_grf(kernel, noise=None, seed=42, npoints=40):
    """                                                                                                                                  
    Function to generate a 1D gaussian random field for a                                                                                
    given scikit-learn kernel.                                                                                                           
                                                                                                                                         
    :param kernel:  given sklearn kernel.                                                                                                
    :param noise:   float. Level of noise to add to the                                                                                  
                    gaussian randomn field. (default: None)                                                                              
    :param seed:    int. seed of the random process. (default: 42)                                                                       
    :param npoints: int. number of points to generate for the                                                                            
                    simulations.                                                                                                         
    """
    # fixing the seed                                                                                                                    
    np.random.seed(seed)
    # generate random 2D coordinate                                                                                                      
    x1 = np.random.uniform(-10,10, npoints)
    x2 = np.random.uniform(-10,10, npoints)
    x = np.array([x1, x2]).T
    # creating the correlation matrix / kernel                                                                                           
    K = kernel.__call__(x)
    # generating gaussian random field                                                                                                   
    y = np.random.multivariate_normal(np.zeros(npoints), K)
    if noise is not None:
        # adding noise                                                                                                                   
        y += np.random.normal(scale=noise, size=npoints)
        y_err = np.ones_like(y) * noise
        return x, y, y_err
    else:
        return x, y, None


def get_correlation_length_matrix(size, e1, e2):
    """                                                                                                                                  
    Produce correlation matrix to introduce anisotropy in kernel.                                                                        
    Used same parametrization as shape measurement in weak-lensing                                                                       
    because this is mathematicaly equivalent (anistropic kernel                                                                          
    will have an elliptical shape).                                                                                                      
                                                                                                                                         
    :param correlation_length: Correlation lenght of the kernel.                                                                         
    :param g1, g2:             Shear applied to isotropic kernel.                                                                        
    """
    if abs(e1)>1:
        raise ValueError('abs(e1) must be lower than 1 (current %f)'%(e1))
    if abs(e2)>1:
        raise ValueError('abs(e2) must be lower than 1 (current %f)'%(e2))
    e = np.sqrt(e1**2 + e2**2)
    q = (1-e) / (1+e)
    phi = 0.5 * np.arctan2(e2,e1)
    rot = np.array([[np.cos(phi), np.sin(phi)],
                    [-np.sin(phi), np.cos(phi)]])
    ell = np.array([[size**2, 0],
                    [0, (size * q)**2]])
    L = np.dot(rot.T, ell.dot(rot))
    return L
    
# Here I am generating the GRF, however it takes some time so I saved the data in a pkl files. 
# However, feel free to play with it.
l = 0.2
L = get_correlation_length_matrix(l, 0., 0.)
invLam = np.linalg.inv(L)
sigma = 1.
kernel = sigma**2 * AnisotropicRBF(invLam=invLam)
if False:
    x, y, y_err = make_2d_grf(kernel, noise=None, seed=42, npoints=10000)
else:
    dic = pickle.load(open('../data/Statistical_uncertainties_on_2PCF_data.pkl', 'rb'))
    x, y, y_err = dic['x'], dic['y'], dic['y_err']

Here some functions to compute the measured 2-PCF (using `treecorr`) and the true kernel 
(using some non obvious manipulation from `sklearn`) + function to plot all of that plus residuals of 2-PCF 

In [3]:
def comp_2pcf_treecorr(kernel, x, y, Filter):

    cat = treecorr.Catalog(x=x[:,0][Filter], y=x[:,1][Filter], k=y[Filter], w=None)
    kk = treecorr.KKCorrelation(min_sep=0, max_sep=1., nbins=17,
                                bin_type='TwoD', bin_slop=0)
    kk.process(cat)

    npixels = len(kk.xi)**2
    coord = np.array([kk.dx.reshape(npixels), kk.dy.reshape(npixels)]).T
    pcf = kernel.__call__(coord,Y=np.zeros_like(coord))[:,0]
    pcf = pcf.reshape((len(kk.xi), len(kk.xi)))
    
    return kk, pcf

def plot_2pcf_out(kk, pcf, MAX = 1**2):
    
    EXT = [np.min(kk.dx), np.max(kk.dx),
           np.min(kk.dy), np.max(kk.dy)]
    CM = plt.cm.seismic

    plt.figure(figsize=(14,5))
    plt.subplots_adjust(wspace=0.5,left=0.07,right=0.95, bottom=0.15,top=0.85)

    plt.subplot(1,3,1)
    plt.imshow(kk.xi, extent=EXT, interpolation='nearest', origin='lower',
               vmin=-MAX, vmax=MAX, cmap=CM)
    cbar = plt.colorbar()
    cbar.update_ticks()
    cbar.set_label('$\\xi$',fontsize=20)
    plt.xlabel('$\Delta X_0$',fontsize=20)
    plt.ylabel('$\Delta X_1$',fontsize=20)
    plt.title('Measured 2-PCF',fontsize=16)

    plt.subplot(1,3,2)
    plt.imshow(pcf, extent=EXT, interpolation='nearest',
               origin='lower',vmin=-MAX,vmax=MAX, cmap=CM)
    cbar = plt.colorbar()
    cbar.formatter.set_powerlimits((0, 0))
    cbar.update_ticks()
    cbar.set_label('$\\xi\'$',fontsize=20)
    plt.xlabel('$\Delta X_0$',fontsize=20)
    plt.title('True 2-PCF',fontsize=16)

    plt.subplot(1,3,3)
    plt.imshow(kk.xi - pcf, extent=EXT, interpolation='nearest',
               origin='lower',vmin=-MAX/2,vmax=MAX/2, cmap=CM)
    cbar = plt.colorbar()
    cbar.formatter.set_powerlimits((0, 0))
    cbar.update_ticks()
    cbar.set_label('$\\xi - \\xi\'$',fontsize=20)
    plt.xlabel('$\Delta X_0$',fontsize=20)
    plt.title('Residuals',fontsize=16)

Here is the interesting function :) 
    
I am plotting a single realization of a gaussian random field, the measured 2-point correlation function of it, the true kernel used to generate the Gaussian random field and the difference between the measured 2-point correlation function and the True Kernel.

I am letting an option of computing the measured 2-point correlation within a given radius, like that it is possible to increase/decrease the radius to increase/decrease the number of data used to compute the 2-point correlation function. 

Something important here, there is no error: the field is "perfect". Only statistical uncertainty occurs while computing the 2-point correlation function as you will see. 

I will not explain here and let you play with this interactive function. Pay attention to how the residuals evolve as you put more data to compute the 2-point correlation function, and let's discuss that tomorrow :) 

In [4]:
@interact(Radius_filter = widgets.FloatSlider(value=1., min=1, max=14., step=0.2, description='$R$:',
          disabled=False,
          continuous_update=False,
          orientation='horizontal',
          readout=True,
          readout_format='.2f'))
def example_covariance_of_2PCF(Radius_filter):

    Filter = np.sqrt(x[:,0]**2 + x[:,1]**2) < Radius_filter

    plt.figure(figsize=(7,6))
    plt.scatter(x[:,0][~Filter], x[:,1][~Filter], s=1, alpha=0.01)
    plt.scatter(x[:,0][Filter], x[:,1][Filter], c=y[Filter], cmap=plt.cm.seismic, vmin=-2*sigma, vmax=2*sigma, s=3)
    cb = plt.colorbar()
    plt.axis('equal')
    plt.xlabel('$X_0$', fontsize=16)
    plt.ylabel('$X_1$', fontsize=16)
    cb.set_label('$Y$', fontsize=16)

    kk, pcf = comp_2pcf_treecorr(kernel, x, y, Filter)

    plot_2pcf_out(kk, pcf, MAX = sigma**2)

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='$R$:', max=14.0, min=1.0, s…